# Training & Evaluation Notebook

Use this after datasets are prepared. Runs training, evaluation, TensorBoard.

In [ ]:
# Detect Colab
try:
    import google.colab  # type: ignore
    IN_COLAB = True
    print('Running on Google Colab')
except ImportError:
    IN_COLAB = False
    print('Running locally')


In [ ]:
# Mount Drive and set WORK_DIR (adjust if you used a different folder)
import os, sys
from pathlib import Path

if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=False)
    WORK_DIR = '/content/drive/MyDrive/handwash_training'
else:
    WORK_DIR = '.'

print('WORK_DIR:', WORK_DIR)


In [ ]:
# Ensure repo root and import training modules
import os, sys
from pathlib import Path

def ensure_repo_root():
    candidates = [
        Path(WORK_DIR) / 'edgeWash',
        Path('/content/edgeWash'),
        Path('/content/drive/MyDrive/edgeWash'),
        Path('/content/drive/MyDrive/handwash_training/edgeWash'),
        Path('/content/drive/MyDrive/handwash_training_colab/edgeWash'),
        Path.cwd()
    ]
    for c in candidates:
        if (c / 'training' / 'config.py').exists():
            os.chdir(c)
            td = c / 'training'
            if str(td) not in sys.path:
                sys.path.insert(0, str(td))
            print('Using repo root:', c)
            print('Added to sys.path:', td)
            return c, td
    raise FileNotFoundError('Cannot locate training/config.py; update WORK_DIR or clone repo.')

repo_root, training_dir = ensure_repo_root()

import config
import download_datasets
import preprocess_data
import data_generators
import models
import train as train_module
import evaluate

print('Training modules imported successfully!')


## Model Options (2D, 3D, and temporal)

Pick a backbone: frame-based (MobileNetV2/ResNet50/EfficientNetB0) or temporal (LSTM/GRU/3D CNN).


In [ ]:
AVAILABLE_MODELS = ['mobilenetv2', 'resnet50', 'efficientnetb0', 'lstm', 'gru', '3d_cnn']
FRAME_BACKBONES = ['mobilenetv2', 'resnet50', 'efficientnetb0']
TEMPORAL_BACKBONES = ['lstm', 'gru', '3d_cnn']
print('Available models:', ', '.join(AVAILABLE_MODELS))
print('Frame models:', ', '.join(FRAME_BACKBONES))
print('Temporal models:', ', '.join(TEMPORAL_BACKBONES))

## Sampling frequency (frame skip)

Set how many frames to skip when extracting. If you change this, rerun preprocessing so splits update.


In [ ]:
FRAME_SKIP_TO_USE = 2  # e.g., 1=all frames, 2=every other, 4=every 4th
config.FRAME_SKIP = FRAME_SKIP_TO_USE
print('Frame skip set to', config.FRAME_SKIP)
print('Available presets:', getattr(config, 'FRAME_SKIP_OPTIONS', [1,2,4]))

## Live TensorBoard (start before training)

Starts TensorBoard so you can watch training live in this notebook.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir {config.LOGS_DIR}

## 7. Train Selected Models

Train any combination of models (2D frame, temporal, 3D CNN). Use the config cell below to pick them.

In [ ]:
# Training configuration
EPOCHS = 10  # adjust as needed
MODELS_TO_TRAIN = ['mobilenetv2', 'resnet50', 'efficientnetb0', 'lstm', 'gru', '3d_cnn']
FRAME_SKIP_USED = config.FRAME_SKIP

print('=' * 80)
print('TRAINING PIPELINE')
print('=' * 80)
print('
Models: ' + ', '.join([m.upper() for m in MODELS_TO_TRAIN]))
print(f'Epochs: {EPOCHS}')
print(f'Frame skip: {FRAME_SKIP_USED}')
print(f'Checkpoints will be saved to: {config.CHECKPOINTS_DIR}')
print(f'Final models will be saved to: {config.MODELS_DIR}')
print('
' + '=' * 80)

In [ ]:
# Train all models
training_results = {}
total_models = len(MODELS_TO_TRAIN)

for idx, model_type in enumerate(MODELS_TO_TRAIN, start=1):
    print('
' + '='*80)
    print(f'TRAINING MODEL {idx}/{total_models}: {model_type.upper()}')
    print('='*80)
    
    if model_type in FRAME_BACKBONES:
        batch_size = 32
    elif model_type == "3d_cnn":
        batch_size = 12
    else:
        batch_size = 16
    
    result = train_module.train_model(
        model_type=model_type,
        train_csv=config.PROCESSED_DIR / 'train.csv',
        val_csv=config.PROCESSED_DIR / 'val.csv',
        batch_size=batch_size,
        epochs=EPOCHS,
        learning_rate=config.LEARNING_RATE
    )
    training_results[model_type] = result
    
    best_epoch = result["best_epoch"]
    best_val_acc = result["history"]["val_accuracy"][best_epoch]
    best_val_loss = result["history"]["val_loss"][best_epoch]
    
    print(f'\n✓ {model_type.upper()} training complete!')
    print(f'  Best epoch: {best_epoch + 1}')
    print(f'  Best val accuracy: {best_val_acc:.4f}')
    print(f'  Best val loss: {best_val_loss:.4f}')
    print('  Final model saved: {}'.format(result['final_model_path']))


## 8. Training Visualization

Compare training curves across all models.

## 9. Evaluate All Models

Evaluate all trained models on test set.

In [ ]:
# Evaluate all models
evaluation_results = {}

print("=" * 80)
print("EVALUATING ALL MODELS ON TEST SET")
print("=" * 80)

for model_type in MODELS_TO_TRAIN:
    print(f"\nEvaluating {model_type.upper()}...")
    
    batch_size = 32 if model_type == 'mobilenetv2' else 16
    
    eval_results = evaluate.evaluate_model(
        model_path=training_results[model_type]['final_model_path'],
        test_csv=config.PROCESSED_DIR / 'test.csv',
        model_type=model_type,
        batch_size=batch_size,
        save_results=True
    )
    
    evaluation_results[model_type] = eval_results
    
    print(f"✓ {model_type.upper()} evaluation complete!")
    print(f"  Accuracy: {eval_results['accuracy']:.4f}")
    print(f"  F1-Score: {eval_results['f1_score']:.4f}")

print("\n" + "=" * 80)
print("ALL EVALUATIONS COMPLETE")
print("=" * 80)

In [ ]:
# Display detailed metrics for each model
for model_type in MODELS_TO_TRAIN:
    eval_results = evaluation_results[model_type]
    
    print("\n" + "=" * 80)
    print(f"{model_type.upper()} - TEST SET METRICS")
    print("=" * 80)
    
    print(f"\nOverall Metrics:")
    print(f"  Accuracy:       {eval_results['accuracy']:.4f}")
    print(f"  Top-2 Accuracy: {eval_results['top2_accuracy']:.4f}")
    print(f"  Precision:      {eval_results['precision']:.4f}")
    print(f"  Recall:         {eval_results['recall']:.4f}")
    print(f"  F1-Score:       {eval_results['f1_score']:.4f}")
    
    print(f"\nPer-Class F1-Scores:")
    for class_name in config.CLASS_NAMES:
        metrics = eval_results['per_class_metrics'][class_name]
        print(f"  {class_name}: {metrics['f1-score']:.4f}")

## 10. TensorBoard

Launch TensorBoard to view training logs.

In [ ]:
# Load TensorBoard extension (Jupyter/Colab)
%load_ext tensorboard

In [ ]:
# Launch TensorBoard
%tensorboard --logdir {config.LOGS_DIR}

## 11. Model Comparison

Compare all 3 models with comprehensive visualizations.

In [ ]:
# Create model comparison visualization
print("=" * 80)
print("GENERATING MODEL COMPARISON PLOTS")
print("=" * 80)

# Call compare_models from evaluate module
comparison_path = config.RESULTS_DIR / 'model_comparison.png'
evaluate.compare_models(
    evaluation_results,
    save_path=comparison_path
)

print(f"\n✓ Comparison plot saved: {comparison_path}")

In [ ]:
# Display comparison plot
from IPython.display import Image, display

if comparison_path.exists():
    display(Image(filename=str(comparison_path)))
else:
    print("Comparison plot not found!")

In [ ]:
# Create summary table
import pandas as pd

summary_data = []
for model_type in MODELS_TO_TRAIN:
    eval_results = evaluation_results[model_type]
    summary_data.append({
        'Model': model_type.upper(),
        'Accuracy': f"{eval_results['accuracy']:.4f}",
        'Top-2 Acc': f"{eval_results['top2_accuracy']:.4f}",
        'Precision': f"{eval_results['precision']:.4f}",
        'Recall': f"{eval_results['recall']:.4f}",
        'F1-Score': f"{eval_results['f1_score']:.4f}"
    })

summary_df = pd.DataFrame(summary_data)

print("\n" + "=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)
print("\n" + summary_df.to_string(index=False))

# Save summary
summary_path = config.RESULTS_DIR / 'model_comparison_summary.csv'
summary_df.to_csv(summary_path, index=False)
print(f"\n✓ Summary saved: {summary_path}")

In [ ]:
# Identify best model
best_model = max(evaluation_results.items(), key=lambda x: x[1]['f1_score'])
best_model_name = best_model[0]
best_f1 = best_model[1]['f1_score']

print("\n" + "=" * 80)
print("BEST MODEL")
print("=" * 80)
print(f"\n🏆 {best_model_name.upper()} achieved the highest F1-Score: {best_f1:.4f}")
print(f"\nAll metrics for {best_model_name.upper()}:")
for metric, value in best_model[1].items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.4f}")

## 12. Saved Models & Checkpoints

Summary of all saved model weights and checkpoints on Google Drive.

In [ ]:
# Display saved model paths
print("=" * 80)
print("SAVED MODEL WEIGHTS (Google Drive)")
print("=" * 80)

print(f"\nModels directory: {config.MODELS_DIR}")
print(f"Checkpoints directory: {config.CHECKPOINTS_DIR}")

print("\nFinal Model Weights:")
for model_type in MODELS_TO_TRAIN:
    model_path = training_results[model_type]['final_model_path']
    checkpoint_path = training_results[model_type]['best_checkpoint_path']
    
    print(f"\n{model_type.upper()}:")
    print(f"  Final model: {model_path}")
    print(f"  Best checkpoint: {checkpoint_path}")
    
    # Check file size
    if Path(model_path).exists():
        size_mb = Path(model_path).stat().st_size / (1024 * 1024)
        print(f"  Model size: {size_mb:.2f} MB")

print("\n" + "=" * 80)
print("All model weights are saved to Google Drive!")
print("They will persist even if Colab runtime disconnects.")
print("=" * 80)

## 13. Summary & Next Steps

Complete training pipeline finished!

In [ ]:
# Training configuration
EPOCHS = 10  # adjust as needed
MODELS_TO_TRAIN = ['mobilenetv2', 'resnet50', 'efficientnetb0', 'lstm', 'gru', '3d_cnn']
FRAME_SKIP_USED = config.FRAME_SKIP

print('=' * 80)
print('TRAINING PIPELINE')
print('=' * 80)
print('
Models: ' + ', '.join([m.upper() for m in MODELS_TO_TRAIN]))
print(f'Epochs: {EPOCHS}')
print(f'Frame skip: {FRAME_SKIP_USED}')
print(f'Checkpoints will be saved to: {config.CHECKPOINTS_DIR}')
print(f'Final models will be saved to: {config.MODELS_DIR}')
print('
' + '=' * 80)

In [ ]:
# Launch TensorBoard
%tensorboard --logdir {config.LOGS_DIR}

## 11. Optional: Train Additional Models

Train LSTM or GRU models for temporal modeling (requires sequence data).

In [ ]:
# Uncomment to train LSTM model

# lstm_result = train_module.train_model(
#     model_type='lstm',
#     train_csv=config.PROCESSED_DIR / 'train.csv',
#     val_csv=config.PROCESSED_DIR / 'val.csv',
#     batch_size=16,  # Reduce batch size for sequence models
#     epochs=20,
#     learning_rate=config.LEARNING_RATE
# )

# print("\n✓ LSTM training complete!")

In [ ]:
# Uncomment to train GRU model

# gru_result = train_module.train_model(
#     model_type='gru',
#     train_csv=config.PROCESSED_DIR / 'train.csv',
#     val_csv=config.PROCESSED_DIR / 'val.csv',
#     batch_size=16,
#     epochs=20,
#     learning_rate=config.LEARNING_RATE
# )

# print("\n✓ GRU training complete!")

## 12. Model Comparison

Compare multiple models (if trained).

In [ ]:
# Example: Compare MobileNetV2, LSTM, GRU
# Uncomment if you trained multiple models

# model_results = {
#     'MobileNetV2': eval_results,
#     'LSTM': evaluate.evaluate_model(
#         model_path=str(config.MODELS_DIR / 'lstm_final.keras'),
#         test_csv=config.PROCESSED_DIR / 'test.csv',
#         model_type='lstm',
#         batch_size=16,
#         save_results=True
#     ),
#     'GRU': evaluate.evaluate_model(
#         model_path=str(config.MODELS_DIR / 'gru_final.keras'),
#         test_csv=config.PROCESSED_DIR / 'test.csv',
#         model_type='gru',
#         batch_size=16,
#         save_results=True
#     )
# }

# # Create comparison plot
# evaluate.compare_models(
#     model_results,
#     save_path=config.RESULTS_DIR / 'model_comparison.png'
# )

# display(Image(filename=str(config.RESULTS_DIR / 'model_comparison.png')))

## 13. Summary & Next Steps

Training pipeline complete!

In [ ]:
# Training configuration
EPOCHS = 10  # adjust as needed
MODELS_TO_TRAIN = ['mobilenetv2', 'resnet50', 'efficientnetb0', 'lstm', 'gru', '3d_cnn']
FRAME_SKIP_USED = config.FRAME_SKIP

print('=' * 80)
print('TRAINING PIPELINE')
print('=' * 80)
print('
Models: ' + ', '.join([m.upper() for m in MODELS_TO_TRAIN]))
print(f'Epochs: {EPOCHS}')
print(f'Frame skip: {FRAME_SKIP_USED}')
print(f'Checkpoints will be saved to: {config.CHECKPOINTS_DIR}')
print(f'Final models will be saved to: {config.MODELS_DIR}')
print('
' + '=' * 80)

## Space-Saving Sequential Pipeline

Download one dataset at a time, train/evaluate, then delete raw/extracted frames to conserve Drive space. Models/logs/results are kept.

In [ ]:
# Configure per-dataset run
DATASETS = ['kaggle', 'pskus', 'metc']  # edit list/order
MODELS = ['mobilenetv2']  # add resnet50, efficientnetb0, lstm, gru, 3d_cnn
EPOCHS = 5
BATCH_FRAME = 32
BATCH_SEQ = 16

from pathlib import Path
import shutil
import train as train_module
import evaluate
import download_datasets
import preprocess_data
import config

def cleanup_dataset(name):
    raw_path = Path('datasets/raw') / name
    if raw_path.exists():
        shutil.rmtree(raw_path, ignore_errors=True)
        print(f'Removed raw: {raw_path}')
    for jpg in Path('datasets/processed').rglob('*.jpg'):
        try:
            jpg.unlink()
        except Exception:
            pass
    for d in sorted(Path('datasets/processed').rglob('*'), key=lambda x: len(str(x)), reverse=True):
        if d.is_dir():
            try:
                next(d.iterdir())
            except StopIteration:
                d.rmdir()

for ds in DATASETS:
    print(f'=== DATASET {ds.upper()} ===')
    if ds == 'kaggle':
        ok = download_datasets.download_kaggle_dataset()
    elif ds == 'pskus':
        ok = download_datasets.download_pskus_dataset()
    elif ds == 'metc':
        ok = download_datasets.download_metc_dataset()
    else:
        raise ValueError(f'Unknown dataset {ds}')
    if not ok:
        print(f'Skipping {ds}, download failed.')
        continue
    result = preprocess_data.preprocess_all_datasets(
        use_kaggle=ds=='kaggle', use_pskus=ds=='pskus', use_metc=ds=='metc')
    if not result:
        print(f'Skipping {ds}, preprocess failed.')
        cleanup_dataset(ds)
        continue
    for model in MODELS:
        batch = BATCH_FRAME if model in ['mobilenetv2','resnet50','efficientnetb0'] else (12 if model=='3d_cnn' else BATCH_SEQ)
        res = train_module.train_model(
            model_type=model,
            train_csv=Path('datasets/processed/train.csv'),
            val_csv=Path('datasets/processed/val.csv'),
            batch_size=batch,
            epochs=EPOCHS,
            learning_rate=config.LEARNING_RATE
        )
        eval_res = evaluate.evaluate_model(
            model_path=res['final_model_path'],
            test_csv=Path('datasets/processed/test.csv'),
            model_type=model,
            batch_size=batch,
            save_results=True
        )
        val_acc = res['history']['val_accuracy'][res['best_epoch']]
        print('{} {}: val_acc={:.4f} test_acc={:.4f}'.format(ds, model, val_acc, eval_res['accuracy']))
    cleanup_dataset(ds)
    print(f'=== DONE {ds.upper()} ===
')
print('Sequential pipeline complete.')
